# Benchmarking Kronecker.jl

In this notebook, we illustrate the basic functionality of `Kronecker.jl` and compare the runtime with native Julia code.

In [1]:
using Kronecker, BenchmarkTools, LinearAlgebra

## Basic functionality

In [2]:
# define some matrices

A = randn(4, 4);
B = Array{Float64, 2}([1 2 3;
     4 5 6;
     7 -2 9]);
C = rand(5, 6);

v = rand(12);

A Kronecker product between two matrices (of the same type) is constructed using $\otimes$, typed as `\otimes TAB`. It is **not** directly evaluated unless we use `collect`.

In [3]:
K = A ⊗ B

Kronecker system of the form A ⊗ B


In [4]:
collect(K)

12×12 Array{Float64,2}:
  0.633645   1.26729    1.90094  …   1.11148    2.22296    3.33445 
  2.53458    3.16823    3.80187      4.44593    5.55741    6.66889 
  4.43552   -1.26729    5.70281      7.78038   -2.22296   10.0033  
  1.33125    2.66249    3.99374      0.711448   1.4229     2.13434 
  5.32499    6.65624    7.98748      2.84579    3.55724    4.26869 
  9.31873   -2.66249   11.9812   …   4.98013   -1.4229     6.40303 
 -0.335013  -0.670026  -1.00504     -0.229921  -0.459843  -0.689764
 -1.34005   -1.67507   -2.01008     -0.919685  -1.14961   -1.37953 
 -2.34509    0.670026  -3.01512     -1.60945    0.459843  -2.06929 
 -0.753235  -1.50647   -2.25971     -0.483323  -0.966645  -1.44997 
 -3.01294   -3.76618   -4.51941  …  -1.93329   -2.41661   -2.89994 
 -5.27265    1.50647   -6.77912     -3.38326    0.966645  -4.3499  

In [5]:
K[3, 4]  # indexing works, computes result directly

10.622391602934137

Compare with Julia native result.

In [6]:
X = kron(A, B)

12×12 Array{Float64,2}:
  0.633645   1.26729    1.90094  …   1.11148    2.22296    3.33445 
  2.53458    3.16823    3.80187      4.44593    5.55741    6.66889 
  4.43552   -1.26729    5.70281      7.78038   -2.22296   10.0033  
  1.33125    2.66249    3.99374      0.711448   1.4229     2.13434 
  5.32499    6.65624    7.98748      2.84579    3.55724    4.26869 
  9.31873   -2.66249   11.9812   …   4.98013   -1.4229     6.40303 
 -0.335013  -0.670026  -1.00504     -0.229921  -0.459843  -0.689764
 -1.34005   -1.67507   -2.01008     -0.919685  -1.14961   -1.37953 
 -2.34509    0.670026  -3.01512     -1.60945    0.459843  -2.06929 
 -0.753235  -1.50647   -2.25971     -0.483323  -0.966645  -1.44997 
 -3.01294   -3.76618   -4.51941  …  -1.93329   -2.41661   -2.89994 
 -5.27265    1.50647   -6.77912     -3.38326    0.966645  -4.3499  

Basic algebraic properties can be computed, but much faster. Note that we compare using the **precomputed** Kronecker product, something which is not even possible for realistic matrices.

Compare:

### Trace

In [7]:
@btime tr($A ⊗ $B)

  14.246 ns (1 allocation: 32 bytes)


29.8995111604951

In [8]:
@btime tr($X)

  10.166 ns (0 allocations: 0 bytes)


29.89951116049511

### Determinant

In [9]:
@btime det($A ⊗ $B)

  509.036 ns (7 allocations: 688 bytes)


3.2593831590633774e6

In [10]:
@btime det($X)

  1.120 μs (3 allocations: 1.42 KiB)


3.259383159063375e6

### Matrix inverse

In [11]:
@btime inv($A ⊗ $B)

  1.162 μs (11 allocations: 4.52 KiB)


Kronecker system of the form A ⊗ B


In [12]:
@btime collect(inv($A ⊗ $B))  # inv returns an implict Kronecker structure

  1.391 μs (11 allocations: 5.70 KiB)


12×12 Array{Float64,2}:
  0.542261   -0.228321   -0.0285401   …  -1.36177    0.573377   0.0716721
  0.0570801  -0.11416     0.0570801      -0.143344   0.286689  -0.143344 
 -0.409074    0.152214   -0.0285401       1.0273    -0.382251   0.0716721
 -0.226558    0.095393    0.0119241      -1.83102    0.770955   0.0963694
 -0.0238482   0.0476965  -0.0238482      -0.192739   0.385478  -0.192739 
  0.170912   -0.0635953   0.0119241   …   1.3813    -0.51397    0.0963694
  0.111836   -0.0470889  -0.00588611      1.33133   -0.560561  -0.0700701
  0.0117722  -0.0235444   0.0117722       0.14014   -0.28028    0.14014  
 -0.0843676   0.0313926  -0.00588611     -1.00434    0.373707  -0.0700701
 -0.903108    0.380256    0.047532        2.69798   -1.13599   -0.141999 
 -0.095064    0.190128   -0.095064    …   0.283998  -0.567996   0.283998 
  0.681292   -0.253504    0.047532       -2.03532    0.757328  -0.141999 

In [13]:
@btime inv($X)

  2.407 μs (5 allocations: 7.58 KiB)


12×12 Array{Float64,2}:
  0.542261   -0.228321   -0.0285401   …  -1.36177    0.573377   0.0716721
  0.0570801  -0.11416     0.0570801      -0.143344   0.286689  -0.143344 
 -0.409074    0.152214   -0.0285401       1.0273    -0.382251   0.0716721
 -0.226558    0.095393    0.0119241      -1.83102    0.770955   0.0963694
 -0.0238482   0.0476965  -0.0238482      -0.192739   0.385478  -0.192739 
  0.170912   -0.0635953   0.0119241   …   1.3813    -0.51397    0.0963694
  0.111836   -0.0470889  -0.00588611      1.33133   -0.560561  -0.0700701
  0.0117722  -0.0235444   0.0117722       0.14014   -0.28028    0.14014  
 -0.0843676   0.0313926  -0.00588611     -1.00434    0.373707  -0.0700701
 -0.903108    0.380256    0.047532        2.69798   -1.13599   -0.141999 
 -0.095064    0.190128   -0.095064    …   0.283998  -0.567996   0.283998 
  0.681292   -0.253504    0.047532       -2.03532    0.757328  -0.141999 

### Matrix vector multiplication

One of the most important features is the efficient multiplication of a Kronecker product with a vector.

In [14]:
@btime ($A ⊗ $B) * $v

  416.724 ns (9 allocations: 784 bytes)


12-element Array{Float64,1}:
  13.986751198142887
  34.08131408780208 
  31.81037680949581 
  14.71191622834013 
  35.98968239708811 
  35.09028642101278 
   4.114222926226501
   9.343592476134516
   5.834526736296128
  -8.89737205315483 
 -21.510052997293325
 -19.204576545907344

In [15]:
@btime kron($A, $B) * $v

  328.929 ns (2 allocations: 1.39 KiB)


12-element Array{Float64,1}:
  13.986751198142885
  34.08131408780207 
  31.810376809495807
  14.71191622834013 
  35.98968239708811 
  35.09028642101278 
   4.114222926226501
   9.343592476134514
   5.834526736296127
  -8.89737205315483 
 -21.51005299729332 
 -19.20457654590734 

Using a somewhat larger example...

In [16]:
C = rand(50, 60);
D = randn(80, 50);
u = collect(1.0:(50*60));

In [17]:
@btime (C ⊗ D) * u;

  25.674 μs (12 allocations: 82.55 KiB)


In [18]:
@btime kron(C, D) * u;

  24.001 ms (4 allocations: 91.58 MiB)


In [19]:
all(((C ⊗ D) * u) ≈ (kron(C, D) * u))

true

## Indexed Krocker systems

In many applications, one has to deal with incomplete Kronecker systems. Indexed systems are supported and Kronecker system-vector multiplications are supported using the generalized vec trick.

In [20]:
v = rand(10);

# sizes
a, b = 4, 8;
c, d = 5, 9;

# matrices
M = randn(a, b);
N = rand(c, d);

# indices
p = rand(1:a, 6);
q = rand(1:c, 6);

r = rand(1:b, 10);
t = rand(1:d, 10);

In [21]:
kronprod = N ⊗ M
ikp = kronprod[p,q,r,t]  # subpart of the Kronecker system

Indexed Kronecker system of the form A ⊗ B


In [22]:
collect(ikp)

6×10 Array{Float64,2}:
 -0.105716  -0.423239  -0.105716  …   0.643765  -0.903264  -0.319691
 -0.170159  -0.490947  -0.170159      0.513776  -0.85826   -0.1808  
  0.682622  -0.18825    0.682622      0.450817   1.46083   -0.387904
  0.682622  -0.18825    0.682622      0.450817   1.46083   -0.387904
  1.07427    0.210861   1.07427      -0.258352  -0.164295  -1.93938 
 -0.156488  -0.286323  -0.156488  …   0.16       0.100769   0.372721

In [23]:
# computing this subsystem explicitely
subsystem = kron(N, M)[a * (q .- 1) .+ p, b * (t .- 1) .+ r]

6×10 Array{Float64,2}:
 -0.105716  -0.423239  -0.105716  …   0.643765  -0.903264  -0.319691
 -0.170159  -0.490947  -0.170159      0.513776  -0.85826   -0.1808  
  0.682622  -0.18825    0.682622      0.450817   1.46083   -0.387904
  0.682622  -0.18825    0.682622      0.450817   1.46083   -0.387904
  1.07427    0.210861   1.07427      -0.258352  -0.164295  -1.93938 
 -0.156488  -0.286323  -0.156488  …   0.16       0.100769   0.372721

### Multiplication

In [24]:
@btime $ikp * $v

  601.835 ns (4 allocations: 592 bytes)


6-element Array{Float64,1}:
 -1.6237876844936756 
 -1.9071851212657172 
  1.5701087572512578 
  1.5701087572512578 
 -0.2650864689843326 
 -0.37625001214648723

In [25]:
@btime kron($N, $M)[$a * ($q .- 1) .+ $p, $b * ($t .- 1) .+ $r] * $v

  2.185 μs (10 allocations: 12.94 KiB)


6-element Array{Float64,1}:
 -1.6237876844936756 
 -1.9071851212657172 
  1.5701087572512578 
  1.5701087572512578 
 -0.26508646898433275
 -0.3762500121464872 

In [26]:
@btime $subsystem * $v  # to be fair, if the system fits into memory, it's faster

  94.946 ns (1 allocation: 128 bytes)


6-element Array{Float64,1}:
 -1.6237876844936756 
 -1.9071851212657172 
  1.5701087572512578 
  1.5701087572512578 
 -0.26508646898433275
 -0.3762500121464872 

## Shifted Kronecker systems

In [27]:
# again, define some matrices
# here, we explicitly work with symmetric matrices 

A = Symmetric(rand(10, 10));
B = Symmetric(rand(30, 30));
v = rand(300);

### Solving a shifted Kronecker system

In [28]:
@btime ($A ⊗ $B + 2I) \ $v

  189.431 μs (62 allocations: 66.27 KiB)


300-element Array{Float64,1}:
  0.8232321849019772 
  0.7951809988226676 
  0.22282064789310538
  3.401704847874946  
  3.8348746603825097 
  2.582027956642196  
 -3.7823749520545586 
 -6.320055684648911  
 -1.2135289810401741 
  4.097031456735285  
 -1.5212483984987055 
  3.654014260852317  
  1.497813130100634  
  ⋮                  
 -0.07059806189189016
  1.4630189652880923 
 -1.523118842636403  
 -0.8162581910305616 
 -0.13980640558056648
  0.4319077223549445 
 -0.9310448249588144 
  0.8580958691195064 
  1.1503079375265173 
 -0.6630333389715537 
 -0.6211722009354042 
 -1.2119155813621463 

In [30]:
@btime (kron($A, $B) + 2I) \ $v

  3.786 ms (13209 allocations: 2.67 MiB)


300-element Array{Float64,1}:
  0.8232321849017679 
  0.7951809988227642 
  0.22282064789320355
  3.4017048478748557 
  3.8348746603825177 
  2.582027956642118  
 -3.7823749520547065 
 -6.320055684649038  
 -1.2135289810400134 
  4.097031456735315  
 -1.5212483984985004 
  3.6540142608523047 
  1.4978131301005866 
  ⋮                  
 -0.07059806189187662
  1.4630189652880703 
 -1.523118842636402  
 -0.8162581910305282 
 -0.13980640558050225
  0.4319077223549016 
 -0.9310448249588485 
  0.8580958691195085 
  1.1503079375265124 
 -0.663033338971506  
 -0.6211722009353845 
 -1.21191558136209   